In [12]:
import sys
print(sys.executable)

c:\Users\Playdata\AppData\Local\anaconda3\envs\vectordb_env\python.exe


In [13]:
import sys
print(sys.version)

3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]


In [ ]:
import shutil

shutil.copy("env.txt", ".env")

FileNotFoundError: [Errno 2] No such file or directory: 'env.txt'

In [14]:
# !pip uninstall fitz -y

In [15]:
# !pip install pymupdf

In [ ]:
# !pip install fitz frontend pymupdf pypdf dotenv langchain torch langchain_community transformers datasets
# !pip install "numpy<2"
# !pip install --upgrade pybind11
# !pip install faiss-gpu

In [14]:
import numpy as np
print(np.__version__)

1.26.4


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
import os

path = "pdf_folder_axa"
car_list = os.listdir("./pdf_folder_axa/1_자동차보험")
short_list = os.listdir("./pdf_folder_axa/2_일반보험")
long_list = os.listdir("./pdf_folder_axa/3_장기보험")
long_hurt = []# long_save = []
long_etc = []
for i in long_list:
    if i == "./pdf_folder_axa/1_자동차보험":
        long_etc.append(i)
    else:
        long_hurt.append(i)

# len(file_list), len(car_list)+len(short_list)+len(long_hurt)+len(long_etc)

In [11]:
# len(car_list),len(short_list),len(long_hurt),len(long_save),len(long_etc)

In [17]:
import faiss
import pickle
import numpy as np
import torch
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
# import fitz  # PyMuPDF
import re

# 1️⃣ GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# 자동차 보험

In [19]:
import os
import re
import pickle
import numpy as np
from tqdm import tqdm
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 경로 설정
base_path = "./pdf_folder_axa/"
car_path = os.path.join(base_path, "1_자동차보험")
car_list = os.listdir(car_path)

# 저장 폴더 생성
os.makedirs("faiss_index", exist_ok=True)
os.makedirs("metadata", exist_ok=True)

print(f"총 {len(car_list)}개의 자동차보험 PDF 문서 처리 시작...\n")

for file in tqdm(car_list, desc="자동차 PDF 처리 중", unit="파일"):
    file_path = os.path.join(car_path, file)
    base_filename = os.path.splitext(file)[0]

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 처리 중 에러 발생: {e}")
        continue

    all_texts = []
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)

    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]
    embeddings = []
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)

    embeddings = np.vstack(embeddings).astype(np.float32)

    try:
        res = faiss.StandardGpuResources()
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)
        gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
        gpu_index.add(embeddings)

        faiss_path = f"faiss_index/{base_filename}.faiss"
        faiss.write_index(faiss.index_gpu_to_cpu(gpu_index), faiss_path)
        print(f"{file} → FAISS 저장 완료: {faiss_path}")
    except Exception as e:
        print(f"[FAISS 오류] {file}: {e}")
        continue

    metadata_path = f"metadata/{base_filename}.pkl"
    with open(metadata_path, "wb") as f:
        pickle.dump(split_documents, f)
        print(f"{file} → 메타데이터 저장 완료: {metadata_path}")

print("\n모든 자동차보험 PDF 처리 및 벡터 DB 저장이 완료되었습니다.")


총 12개의 자동차보험 PDF 문서 처리 시작...



자동차 PDF 처리 중:   8%|▊         | 1/12 [2:05:12<22:57:16, 7512.40s/파일]

[FAISS 오류] 001_개인용자동차보험.pdf: module 'faiss' has no attribute 'StandardGpuResources'


자동차 PDF 처리 중:  17%|█▋        | 2/12 [3:05:30<14:30:15, 5221.50s/파일]

[FAISS 오류] 002_개인용자동차보험공동.pdf: module 'faiss' has no attribute 'StandardGpuResources'


자동차 PDF 처리 중:  17%|█▋        | 2/12 [3:49:02<19:05:10, 6871.05s/파일]


KeyboardInterrupt: 

In [20]:
import os
import re
import pickle
import numpy as np
import faiss
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# ✅ 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# ✅ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    pool_mask = torch.ones_like(inputs["input_ids"]).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# ✅ 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# ✅ 경로 설정
base_path = "./pdf_folder_axa/1_자동차보험"
car_list = os.listdir(base_path)

# ✅ 저장 폴더 생성
os.makedirs("faiss_index", exist_ok=True)
os.makedirs("metadata", exist_ok=True)

print(f"총 {len(car_list)}개의 자동차보험 PDF 문서 처리 시작...\n")

for file in tqdm(car_list, desc="자동차 PDF 처리 중", unit="파일"):
    file_path = os.path.join(base_path, file)
    base_filename = os.path.splitext(file)[0]

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 처리 중: {e}")
        continue

    # ✅ 텍스트 정제 및 문서화
    all_texts = [clean_text(doc.page_content) for doc in documents]
    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = [
        Document(page_content=text, metadata={"page": i + 1})
        for i, text in enumerate(unique_texts) if text.strip()
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]
    embeddings = []
    batch_size = 16

    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        try:
            emb = get_embeddings(batch)
            embeddings.append(emb)
        except Exception as e:
            print(f"[임베딩 오류] {file} 배치 실패: {e}")
            continue

    if not embeddings:
        print(f"[건너뜀] {file} → 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    # ✅ FAISS 저장 (CPU용)
    try:
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)

        faiss_path = f"faiss_index/{base_filename}.faiss"
        faiss.write_index(index, faiss_path)
        print(f"{file} → FAISS 저장 완료: {faiss_path}")
    except Exception as e:
        print(f"[FAISS 오류] {file}: {e}")
        continue

    # ✅ 메타데이터 저장
    metadata_path = f"metadata/{base_filename}.pkl"
    with open(metadata_path, "wb") as f:
        pickle.dump(split_documents, f)
        print(f"{file} → 메타데이터 저장 완료: {metadata_path}")

print("\n✅ 모든 PDF 문서 처리 및 벡터 저장이 완료되었습니다.")


총 12개의 자동차보험 PDF 문서 처리 시작...



자동차 PDF 처리 중:   8%|▊         | 1/12 [1:28:13<16:10:25, 5293.20s/파일]

[FAISS 오류] 001_개인용자동차보험.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open faiss_index/001_개인용자동차보험.faiss for writing: Illegal byte sequence


자동차 PDF 처리 중:  17%|█▋        | 2/12 [2:37:46<12:52:25, 4634.51s/파일]

[FAISS 오류] 002_개인용자동차보험공동.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open faiss_index/002_개인용자동차보험공동.faiss for writing: Illegal byte sequence


...

[FAISS 오류] 003_다이렉트개인용자동차보험.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open faiss_index/003_다이렉트개인용자동차보험.faiss for writing: Illegal byte sequence


자동차 PDF 처리 중:  33%|███▎      | 4/12 [17:56:16<38:42:21, 17417.72s/파일]

[FAISS 오류] 004_다이렉트개인용인터넷자동차보험.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open faiss_index/004_다이렉트개인용인터넷자동차보험.faiss for writing: Illegal byte sequence


자동차 PDF 처리 중:  33%|███▎      | 4/12 [18:14:46<36:29:33, 16421.69s/파일]


KeyboardInterrupt: 

In [21]:
import os
import re
import pickle
import numpy as np
import faiss
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 안전한 파일명 변환 함수
def get_safe_filename(name):
    return ''.join(c if c.isalnum() else '_' for c in name)

# 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    pool_mask = torch.ones_like(inputs["input_ids"]).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 경로 설정
base_path = "./pdf_folder_axa/1_자동차보험"
car_list = os.listdir(base_path)

# 저장 폴더 생성
os.makedirs("faiss_index", exist_ok=True)
os.makedirs("metadata", exist_ok=True)

print(f"총 {len(car_list)}개의 자동차보험 PDF 문서 처리 시작...\n")

for file in tqdm(car_list, desc="자동차 PDF 처리 중", unit="파일"):
    file_path = os.path.join(base_path, file)
    base_filename = os.path.splitext(file)[0]
    safe_filename = get_safe_filename(base_filename)

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 처리 중: {e}")
        continue

    # 텍스트 정제 및 문서화
    all_texts = [clean_text(doc.page_content) for doc in documents]
    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = [
        Document(page_content=text, metadata={"page": i + 1, "source": file})
        for i, text in enumerate(unique_texts) if text.strip()
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]
    embeddings = []
    batch_size = 16

    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        try:
            emb = get_embeddings(batch)
            embeddings.append(emb)
        except Exception as e:
            print(f"[임베딩 오류] {file} 배치 실패: {e}")
            continue

    if not embeddings:
        print(f"[건너뜀] {file} → 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    # FAISS 저장 (CPU용)
    try:
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)

        faiss_path = f"faiss_index/{safe_filename}.faiss"
        faiss.write_index(index, faiss_path)
        print(f"{file} → FAISS 저장 완료: {faiss_path}")
    except Exception as e:
        print(f"[FAISS 오류] {file}: {e}")
        continue

    # 메타데이터 저장 (원본 파일명 유지)
    metadata_path = f"metadata/{safe_filename}.pkl"
    with open(metadata_path, "wb") as f:
        pickle.dump(split_documents, f)
        print(f"{file} → 메타데이터 저장 완료: {metadata_path}")

print("\n모든 PDF 문서 처리 및 벡터 저장이 완료되었습니다.")


총 12개의 자동차보험 PDF 문서 처리 시작...



자동차 PDF 처리 중:   8%|▊         | 1/12 [1:50:15<20:12:52, 6615.71s/파일]

[FAISS 오류] 001_개인용자동차보험.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open faiss_index/001_개인용자동차보험.faiss for writing: Illegal byte sequence


자동차 PDF 처리 중:   8%|▊         | 1/12 [2:10:41<23:57:35, 7841.41s/파일]


KeyboardInterrupt: 

In [22]:
import os
import re
import pickle
import numpy as np
import faiss
import torch
import unicodedata
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 안전한 파일명 생성 (영문, 숫자만)
def get_safe_filename(name):
    name = unicodedata.normalize('NFKD', name)
    return ''.join(c if c.isalnum() else '_' for c in name)

# 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    pool_mask = torch.ones_like(inputs["input_ids"]).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 경로 설정
base_pdf_dir = "./pdf_folder_axa/1_자동차보험"
car_list = os.listdir(base_pdf_dir)

# 안전한 저장 디렉토리
faiss_dir = os.path.abspath("faiss_index_safe")
metadata_dir = os.path.abspath("metadata_safe")
os.makedirs(faiss_dir, exist_ok=True)
os.makedirs(metadata_dir, exist_ok=True)

print(f"총 {len(car_list)}개의 자동차보험 PDF 문서 처리 시작...\n")

for file in tqdm(car_list, desc="자동차 PDF 처리 중", unit="파일"):
    file_path = os.path.join(base_pdf_dir, file)
    base_filename = os.path.splitext(file)[0]
    safe_filename = get_safe_filename(base_filename)

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 로드 실패: {e}")
        continue

    # 텍스트 정제 및 문서화
    all_texts = [clean_text(doc.page_content) for doc in documents]
    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = [
        Document(page_content=text, metadata={"page": i + 1, "source": file})
        for i, text in enumerate(unique_texts) if text.strip()
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]
    embeddings = []
    batch_size = 16

    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        try:
            emb = get_embeddings(batch)
            embeddings.append(emb)
        except Exception as e:
            print(f"[임베딩 오류] {file} - 배치 {i // batch_size + 1}: {e}")
            continue

    if not embeddings:
        print(f"[건너뜀] {file} → 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    # FAISS 인덱스 저장
    try:
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)

        faiss_path = os.path.join(faiss_dir, f"{safe_filename}.faiss")
        faiss.write_index(index, faiss_path)
        print(f"{file} → FAISS 저장 완료: {faiss_path}")
    except Exception as e:
        print(f"[FAISS 저장 실패] {file}: {e}")
        continue

    # 메타데이터 저장
    try:
        metadata_path = os.path.join(metadata_dir, f"{safe_filename}.pkl")
        with open(metadata_path, "wb") as f:
            pickle.dump(split_documents, f)
        print(f"{file} → 메타데이터 저장 완료: {metadata_path}")
    except Exception as e:
        print(f"[메타데이터 저장 실패] {file}: {e}")
        continue

print("\n✅ 모든 PDF 문서 처리 및 벡터 저장이 완료되었습니다.")


총 12개의 자동차보험 PDF 문서 처리 시작...



자동차 PDF 처리 중:   8%|▊         | 1/12 [1:36:17<17:39:10, 5777.32s/파일]

[FAISS 저장 실패] 001_개인용자동차보험.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\001_개인용자동차보험.faiss for writing: Illegal byte sequence


자동차 PDF 처리 중:   8%|▊         | 1/12 [1:42:48<18:50:54, 6168.58s/파일]


KeyboardInterrupt: 

In [23]:
import os
import re
import pickle
import hashlib
import numpy as np
import faiss
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 완전 안전한 파일명 생성 (ASCII only + md5 해시)
def get_safe_filename(name):
    name_ascii = re.sub(r'[^\w]', '_', name)  # 영문, 숫자, 밑줄만 남김
    hash_digest = hashlib.md5(name.encode('utf-8')).hexdigest()[:8]
    return f"{name_ascii}_{hash_digest}"

# 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    pool_mask = torch.ones_like(inputs["input_ids"]).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 경로 설정
base_pdf_dir = "./pdf_folder_axa/1_자동차보험"
car_list = os.listdir(base_pdf_dir)

# 안전한 저장 디렉토리
faiss_dir = os.path.abspath("faiss_index_safe")
metadata_dir = os.path.abspath("metadata_safe")
os.makedirs(faiss_dir, exist_ok=True)
os.makedirs(metadata_dir, exist_ok=True)

print(f"총 {len(car_list)}개의 자동차보험 PDF 문서 처리 시작...\n")

for file in tqdm(car_list, desc="자동차 PDF 처리 중", unit="파일"):
    file_path = os.path.join(base_pdf_dir, file)
    base_filename = os.path.splitext(file)[0]
    safe_filename = get_safe_filename(base_filename)

    print(f"파일 처리 중: {file} → 저장 파일명: {safe_filename}.faiss")

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 로드 실패: {e}")
        continue

    # 텍스트 정제 및 문서화
    all_texts = [clean_text(doc.page_content) for doc in documents]
    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = [
        Document(page_content=text, metadata={"page": i + 1, "source": file})
        for i, text in enumerate(unique_texts) if text.strip()
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]
    embeddings = []
    batch_size = 16

    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        try:
            emb = get_embeddings(batch)
            embeddings.append(emb)
        except Exception as e:
            print(f"[임베딩 오류] {file} - 배치 {i // batch_size + 1}: {e}")
            continue

    if not embeddings:
        print(f"[건너뜀] {file} → 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    # FAISS 인덱스 저장
    try:
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)

        faiss_path = os.path.join(faiss_dir, f"{safe_filename}.faiss")
        faiss.write_index(index, faiss_path)
        print(f"{file} → FAISS 저장 완료: {faiss_path}")
    except Exception as e:
        print(f"[FAISS 저장 실패] {file}: {e}")
        continue

    # 메타데이터 저장
    try:
        metadata_path = os.path.join(metadata_dir, f"{safe_filename}.pkl")
        with open(metadata_path, "wb") as f:
            pickle.dump(split_documents, f)
        print(f"{file} → 메타데이터 저장 완료: {metadata_path}")
    except Exception as e:
        print(f"[메타데이터 저장 실패] {file}: {e}")
        continue

print("\n✅ 모든 PDF 문서 처리 및 벡터 저장이 완료되었습니다.")


총 12개의 자동차보험 PDF 문서 처리 시작...



자동차 PDF 처리 중:   0%|          | 0/12 [00:00<?, ?파일/s]

파일 처리 중: 001_개인용자동차보험.pdf → 저장 파일명: 001_개인용자동차보험_08a2926e.faiss


자동차 PDF 처리 중:   8%|▊         | 1/12 [1:32:04<16:52:44, 5524.04s/파일]

[FAISS 저장 실패] 001_개인용자동차보험.pdf: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\001_개인용자동차보험_08a2926e.faiss for writing: Illegal byte sequence
파일 처리 중: 002_개인용자동차보험공동.pdf → 저장 파일명: 002_개인용자동차보험공동_5332a74d.faiss


자동차 PDF 처리 중:   8%|▊         | 1/12 [1:37:52<17:56:33, 5872.17s/파일]


KeyboardInterrupt: 

# AXA faiss DB

In [ ]:
import os
import re
import pickle
import hashlib
import json
import numpy as np
import faiss
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 완전한 ASCII-only safe 파일명 생성
def get_safe_filename(name):
    return "doc_" + hashlib.md5(name.encode("utf-8")).hexdigest()[:8]

# 임베딩 함수
def get_embeddings(texts):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    pool_mask = torch.ones_like(inputs["input_ids"]).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 경로 설정
base_pdf_dir = "./pdf_folder_axa/1_자동차보험"
car_list = os.listdir(base_pdf_dir)

# 저장 디렉토리
faiss_dir = os.path.abspath("faiss_index_axa")
metadata_dir = os.path.abspath("metadata_axa")
os.makedirs(faiss_dir, exist_ok=True)
os.makedirs(metadata_dir, exist_ok=True)

# 매핑 정보 저장용
file_mapping = {}

print(f"총 {len(car_list)}개의 자동차보험 PDF 문서 처리 시작...\n")

for file in tqdm(car_list, desc="자동차 PDF 처리 중", unit="파일"):
    file_path = os.path.join(base_pdf_dir, file)
    base_filename = os.path.splitext(file)[0]
    safe_filename = get_safe_filename(base_filename)

    file_mapping[safe_filename] = base_filename  # 매핑 저장

    print(f"파일 처리 중: {file} → 저장 파일명: {safe_filename}.faiss")

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 로드 실패: {e}")
        continue

    all_texts = [clean_text(doc.page_content) for doc in documents]
    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = [
        Document(page_content=text, metadata={"page": i + 1, "source": file})
        for i, text in enumerate(unique_texts) if text.strip()
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]
    embeddings = []
    batch_size = 16

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        try:
            emb = get_embeddings(batch)
            embeddings.append(emb)
        except Exception as e:
            print(f"[임베딩 오류] {file} - 배치 {i // batch_size + 1}: {e}")
            continue

    if not embeddings:
        print(f"[건너뜀] {file} → 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    # FAISS 저장
    try:
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)

        faiss_path = os.path.join(faiss_dir, f"{safe_filename}.faiss")
        faiss.write_index(index, faiss_path)
        print(f"{file} → FAISS 저장 완료: {faiss_path}")
    except Exception as e:
        print(f"[FAISS 저장 실패] {file}: {e}")
        continue

    # 메타데이터 저장
    try:
        metadata_path = os.path.join(metadata_dir, f"{safe_filename}.pkl")
        with open(metadata_path, "wb") as f:
            pickle.dump(split_documents, f)
        print(f"{file} → 메타데이터 저장 완료: {metadata_path}")
    except Exception as e:
        print(f"[메타데이터 저장 실패] {file}: {e}")
        continue

# 매핑 저장
mapping_path = os.path.join(faiss_dir, "file_mapping.json")
with open(mapping_path, "w", encoding="utf-8") as f:
    json.dump(file_mapping, f, ensure_ascii=False, indent=2)
print(f"\n✅ 파일 매핑 저장 완료: {mapping_path}")

print("\n✅ 모든 PDF 문서 처리 및 벡터 저장이 완료되었습니다.")


총 12개의 자동차보험 PDF 문서 처리 시작...



자동차 PDF 처리 중:   0%|          | 0/12 [00:00<?, ?파일/s]

파일 처리 중: 001_개인용자동차보험.pdf → 저장 파일명: doc_08a2926e.faiss


자동차 PDF 처리 중:   8%|▊         | 1/12 [1:32:28<16:57:12, 5548.37s/파일]

001_개인용자동차보험.pdf → FAISS 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\doc_08a2926e.faiss
001_개인용자동차보험.pdf → 메타데이터 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\metadata_safe\doc_08a2926e.pkl
파일 처리 중: 002_개인용자동차보험공동.pdf → 저장 파일명: doc_5332a74d.faiss


...

010_영업용자동차보험공동.pdf → FAISS 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\doc_fba0b0a0.faiss
010_영업용자동차보험공동.pdf → 메타데이터 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\metadata_safe\doc_fba0b0a0.pkl
파일 처리 중: 011_이륜자동차보험.pdf → 저장 파일명: doc_0a72f811.faiss


자동차 PDF 처리 중:  92%|█████████▏| 11/12 [13:27:00<1:02:02, 3722.50s/파일]

011_이륜자동차보험.pdf → FAISS 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\doc_0a72f811.faiss
011_이륜자동차보험.pdf → 메타데이터 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\metadata_safe\doc_0a72f811.pkl
파일 처리 중: 012_이륜자동차보험공동.pdf → 저장 파일명: doc_f42fcf39.faiss


자동차 PDF 처리 중: 100%|██████████| 12/12 [14:22:32<00:00, 4312.71s/파일]  

012_이륜자동차보험공동.pdf → FAISS 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\doc_f42fcf39.faiss
012_이륜자동차보험공동.pdf → 메타데이터 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\metadata_safe\doc_f42fcf39.pkl

✅ 파일 매핑 저장 완료: c:\Users\Playdata\OneDrive\Desktop\3rd-test\faiss_index_safe\file_mapping.json

✅ 모든 PDF 문서 처리 및 벡터 저장이 완료되었습니다.


# 일반보험

In [7]:
import os
import re
import pickle
import numpy as np
from tqdm import tqdm
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 1️⃣ 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 2️⃣ 안전한 임베딩 생성 함수
def get_embeddings(texts):
    valid_texts = [t for t in texts if isinstance(t, str) and t.strip()]
    if not valid_texts:
        return None

    try:
        inputs = tokenizer(
            valid_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        inputs = {key: val.to(device) for key, val in inputs.items()}
        pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(**inputs, pool_mask=pool_mask)

        return outputs.embedding.cpu().numpy()
    except Exception as e:
        print(f"[임베딩 오류] 배치 토크나이징 중 예외 발생: {e}")
        return None

# 3️⃣ 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 4️⃣ 파일 경로 설정
path = "pdf_folder_axa"
short_list = os.listdir(os.path.join(path, "2_일반보험"))

# 5️⃣ 저장 경로
faiss_index_path = "./faiss_index_axa_short.bin"
metadata_path = "./documents_axa_short.pkl"

# 6️⃣ FAISS GPU 자원 할당 (저장 전에 CPU로 변환할 예정)
res = faiss.StandardGpuResources()
index = None
all_documents = []

print(f"📄 총 {len(short_list)}개의 일반보험 PDF 문서 처리 시작...\n")

for file in tqdm(short_list, desc="PDF 처리 중", unit="파일"):
    all_texts = []
    file_path = os.path.join(path, "2_일반보험", file)

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 로딩 중 문제 발생: {e}")
        continue

    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)

    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]

    embeddings = []
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)

        if batch_embeddings is not None:
            embeddings.append(batch_embeddings)
        else:
            print(f"[스킵] {file} - {i}번째 배치 임베딩 실패")

    if not embeddings:
        print(f"[스킵] {file} - 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)
        id_index = faiss.IndexIDMap(cpu_index)
        index = faiss.index_cpu_to_gpu(res, 0, id_index)  # GPU로 올림

    start_id = index.ntotal
    ids = np.arange(start_id, start_id + embeddings.shape[0]).astype(np.int64)
    index.add_with_ids(embeddings, ids)

    all_documents.extend(split_documents)
    print(f"✅ {file} 처리 완료 — {len(split_documents)}개 문서 생성")

# 7️⃣ FAISS 인덱스 저장 (GPU → CPU 변환 후 저장)
cpu_index = faiss.index_gpu_to_cpu(index)
faiss.write_index(cpu_index, faiss_index_path)
print(f"\n📦 FAISS 인덱스 저장 완료: {faiss_index_path}")

# 8️⃣ 문서 메타데이터 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)
print(f"📝 문서 메타데이터 저장 완료: {metadata_path}")

print("\n🎉 모든 문서 처리 및 저장 완료!")


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

configuration_kanana2vec.py:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

...

✅ 01920250101.pdf 처리 완료 — 341개 문서 생성


PDF 처리 중:  92%|█████████▏| 11/12 [17:57<00:45, 45.40s/파일]

✅ 02220220101.pdf 처리 완료 — 190개 문서 생성


PDF 처리 중: 100%|██████████| 12/12 [18:30<00:00, 92.56s/파일]

✅ 02020250201.pdf 처리 완료 — 277개 문서 생성

📦 FAISS 인덱스 저장 완료: ./faiss_index_axa_short.bin
📝 문서 메타데이터 저장 완료: ./documents_axa_short.pkl

🎉 모든 문서 처리 및 저장 완료!


# 장기보험

In [5]:
import os
import re
import pickle
import numpy as np
from tqdm import tqdm
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 1️⃣ 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 2️⃣ 안전한 임베딩 생성 함수
def get_embeddings(texts):
    valid_texts = [t for t in texts if isinstance(t, str) and t.strip()]
    if not valid_texts:
        return None

    try:
        inputs = tokenizer(
            valid_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        inputs = {key: val.to(device) for key, val in inputs.items()}
        pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(**inputs, pool_mask=pool_mask)

        return outputs.embedding.cpu().numpy()
    except Exception as e:
        print(f"[임베딩 오류] 배치 토크나이징 중 예외 발생: {e}")
        return None

# 3️⃣ 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)
    text = re.sub(r"금소법|법령", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# 4️⃣ 파일 경로 설정
path = "pdf_folder_axa"
car_list = os.listdir(os.path.join(path, "1_자동차보험"))
short_list = os.listdir(os.path.join(path, "2_일반보험"))
long_list = os.listdir(os.path.join(path, "3_장기보험"))

long_hurt = []
long_etc = []


# 5️⃣ 저장 경로
faiss_index_path = "./faiss_index_axa_long.bin"
metadata_path = "./documents_axa_long.pkl"

# 6️⃣ FAISS 인덱스 준비 (저장 가능하도록 IndexIDMap 사용)
index = None
all_documents = []

print(f"📄 총 {len(long_list)}개의 장기보험 PDF 문서 처리 시작...\n")

for file in tqdm(long_list, desc="PDF 처리 중", unit="파일"):
    all_texts = []
    file_path = os.path.join(path, "3_장기보험", file)

    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except Exception as e:
        print(f"[오류] {file} 로딩 중 문제 발생: {e}")
        continue

    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)

    total_text = "\n".join(all_texts)
    unique_texts = list(dict.fromkeys(total_text.split("\n")))

    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)

    texts = [doc.page_content for doc in split_documents]

    embeddings = []
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)

        if batch_embeddings is not None:
            embeddings.append(batch_embeddings)
        else:
            print(f"[스킵] {file} - {i}번째 배치 임베딩 실패")

    if not embeddings:
        print(f"[스킵] {file} - 유효한 임베딩 없음")
        continue

    embeddings = np.vstack(embeddings).astype(np.float32)

    if index is None:
        embedding_dim = embeddings.shape[1]
        base_index = faiss.IndexFlatL2(embedding_dim)
        index = faiss.IndexIDMap(base_index)

    start_id = index.ntotal
    ids = np.arange(start_id, start_id + embeddings.shape[0]).astype(np.int64)
    index.add_with_ids(embeddings, ids)

    all_documents.extend(split_documents)
    print(f"✅ {file} 처리 완료 — {len(split_documents)}개 문서 생성")

# 7️⃣ FAISS 인덱스 저장
faiss.write_index(index, faiss_index_path)
print(f"\n📦 FAISS 인덱스 저장 완료: {faiss_index_path}")

# 8️⃣ 문서 메타데이터 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)
print(f"📝 문서 메타데이터 저장 완료: {metadata_path}")

print("\n🎉 모든 문서 처리 및 저장 완료!")


configuration_kanana2vec.py:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/kakaocorp/kanana-nano-2.1b-embedding:
- configuration_kanana2vec.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_kanana2vec.py:   0%|          | 0.00/9.50k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/kakaocorp/kanana-nano-2.1b-embedding:
- modeling_kanana2vec.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

...

PDF 처리 중:  93%|█████████▎| 14/15 [21:24<01:29, 89.58s/파일]

✅ 02620250101.pdf 처리 완료 — 7661개 문서 생성


PDF 처리 중: 100%|██████████| 15/15 [21:24<00:00, 85.64s/파일]

✅ 02520111001.pdf 처리 완료 — 3개 문서 생성



📦 FAISS 인덱스 저장 완료: ./faiss_index_axa_long.bin
📝 문서 메타데이터 저장 완료: ./documents_axa_long.pkl

🎉 모든 문서 처리 및 저장 완료!


In [ ]:
# 1️⃣ 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index.bin")

# 2️⃣ 문서 정보 로드
with open("./documents.pkl", "rb") as f:
    documents = pickle.load(f)

# 3️⃣ 검색 수행
# query = " 보험금의 종류 및 한도에 대해 설명해줘?"
query = "자동차 의무보험 미가입에 따른 불이익을 알려줘"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=10)  # 가장 유사한 5개 검색
context = []

# 4️⃣ 검색 결과 출력
for idx in I[0]:
    context.append(documents[idx].page_content)
    print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")
